#**Text Generation using GPT-2**

In [ ]:
###### Installing Libraries

! pip install transformers
! pip install tokenizers

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 23.9MB/s 
     |████████████████████████████████| 890kB 24.0MB/s 
     |████████████████████████████████| 3.0MB 44.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=328412f46753e2a0b312e2e107cc810e898c1ca74bb3be119c0d34913cd6550c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
###### Loading Dataset

##### Download dataset
!wget --no-hsts https://github.com/Rutvik-Trivedi/files/blob/master/cleaned_data.tar.xz?raw=true
!mv cleaned_data.tar.xz?raw=true dataset.tar.xz

##### Unzip the Dataset
!tar -xf dataset.tar.xz

--2020-08-10 02:18:46--  https://github.com/Rutvik-Trivedi/files/blob/master/cleaned_data.tar.xz?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Rutvik-Trivedi/files/raw/master/cleaned_data.tar.xz [following]
--2020-08-10 02:18:46--  https://github.com/Rutvik-Trivedi/files/raw/master/cleaned_data.tar.xz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Rutvik-Trivedi/files/master/cleaned_data.tar.xz [following]
--2020-08-10 02:18:47--  https://raw.githubusercontent.com/Rutvik-Trivedi/files/master/cleaned_data.tar.xz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTT

In [ ]:
###### Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import gc
from sklearn.utils import shuffle

In [ ]:
###### Reading Dataset

tweet_set = []

l = os.listdir('cleaned_data/')
for user in l:
  tweets = pd.read_csv('cleaned_data/'+user)
  tweet_set += tweets['tweets'].tolist()

print(len((tweet_set))) # Total Number of tweets

tweet_set = shuffle(tweet_set) # Shuffling the dataset of tweets

with open('tweet.txt', "w") as myfile: # Making a file 'tweet.txt' from present tweets
        for c in tweet_set:
                myfile.write("%s\n" % c)

tweet_set_final = []

for item in tweet_set:
  if(type(item) !=  float):
    tweet_set_final.append(item)

3472574


In [ ]:
###### Training Language Model

from transformers import GPT2Config
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import TFTrainer, TFTrainingArguments
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import tqdm

#### Use of Custom Tokenizer

!mkdir Tweet_GPT2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', do_lower_case=True,add_special_tokens=True,max_length=128,pad_to_max_length=True)
#tokenizer.save_model("Tweet_GPT2")

#config = GPT2Config(vocab_size=25000,max_position_embeddings=514,type_vocab_size=1)
#transformer_model = TFGPT2LMHeadModel.from_pretrained('gpt2',config)

#dataset = LineByLineTextDataset(tokenizer=tokenizer,file_path="/content/tweet.txt",block_size=128)
#data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

#training_args = TrainingArguments(output_dir="/content/Tweet_GPT2",overwrite_output_dir=True,num_train_epochs=1)
#trainer = TFTrainer(model=transformer_model,args=training_args,data_collator=data_collator,train_dataset=dataset,prediction_loss_only=True)

#### Training the Tokenizer

#paths = '/content/tweet.txt'

#tokenizer = ByteLevelBPETokenizer()
#tokenizer.train(files=paths, vocab_size=25000, min_frequency=2, special_tokens=["<s>","<pad>","</s>","<unk>","<mask>",])

#tokenizer = ByteLevelBPETokenizer('/content/Tweet_GPT2/vocab.json','/content/Tweet_GPT2/merges.txt')
#tokenizer._tokenizer.post_processor = BertProcessing(("</s>",tokenizer.token_to_id("</s>")),("s",tokenizer.token_to_id("<s>")),)
#tokenizer.enable_truncation(max_length=512)

#### Use of Pre-Trained Tokenizer

### Defining Tokenizer

#tokenizer = GPT2Tokenizer('/content/Tweet_GPT2/vocab.json','/content/Tweet_GPT2/merges.txt',"gpt2",do_lower_case=True,add_special_tokens=True,max_length=128,pad_to_max_length=True)
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#def tokenize(sentences, tokenizer):
#    input_ids, input_masks = [],[]
#    for sentence in (sentences):
#        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=128, pad_to_max_length=True, return_attention_mask=True)
#        input_ids.append(inputs['input_ids'])
#        input_masks.append(inputs['attention_mask'])

#    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32')

In [ ]:
#### Building the Model

### Defining the Model

config = GPT2Config()
config.output_hidden_states = False
transformer_model = TFGPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)

#input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
#input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
#transformer_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

#model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = transformer_layer)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#### Text Generation through Pre-Trained Model

input_text = tokenizer.encode('Deep Learning involves Neural Networks', return_tensors='tf')
#print(tweet_set[i])

### Greedy Search
greedy_output = transformer_model.generate(input_text,max_length=100)
print(greedy_output)
#print(tokenizer.decode(greedy_output[0],skip_special_tokens=True))

### Beam Search
beam_output = transformer_model.generate(input_text,num_beams=5,max_length=100)
print(beam_output)
#print(tokenizer.decode(beam_output[0],skip_special_tokens=True))

### Beam Search with n-gram Repetition removed
nr_output = transformer_model.generate(input_text,num_beams=5,no_repeat_ngram_size=5,max_length=100)
print(nr_output)
#print(tokenizer.decode(nr_output[0],skip_special_tokens=True))

tf.Tensor(
[[22089  4673  9018 17019  7686   326   389  1498   284  2193   422   262
   5128   290  2193   422   262  5072    13   198   198   464   717  2239
    287  4673   318   284  2193   422   262  5128   290  2193   422   262
   5072    13   383  1218  2239   318   284  2193   422   262  5128   290
   2193   422   262  5072    13   198   198   464   717  2239   287  4673
    318   284  2193   422   262  5128   290  2193   422   262  5072    13
    383  1218  2239   318   284  2193   422   262  5128   290  2193   422
    262  5072    13   198   198   464   717  2239   287  4673   318   284
   2193   422   262  5128]], shape=(1, 100), dtype=int32)
tf.Tensor(
[[22089  4673  9018 17019  7686    13   198   198   818   428  3348    11
    356   905   326 17019  7686   460   307   973   284  2193   284  4331
    262  2003    13   775   905   326 17019  7686   460   307   973   284
   2193   284  4331   262  2003    13   198   198   818   428  3348    11
    356   905   326 17019  7686 

In [ ]:
#### Fine-Tuning the Model

### Dataset Generation
dataset = LineByLineTextDataset(tokenizer=tokenizer,file_path="/content/tweet.txt",block_size=128)

In [ ]:
### Dataset Conversion
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

### Training Argument Generation
training_args = TFTrainingArguments(output_dir="/content/Tweet_GPT2",overwrite_output_dir=True,num_train_epochs=1)
trainer = TFTrainer(model=transformer_model,args=training_args,train_dataset=dataset,prediction_loss_only=True)

### Training the Model
trainer.train()

In [ ]:
### Text Generation through Fine Tuned Model

input_text = tokenizer.encode('Deep Learning involves Neural Networks', return_tensors='tf')
print(tweet_set[1000])

## Greedy Search
greedy_output = transformer_model.generate(input_text,max_length=100)
print(tokenizer.decode(greedy_output[0],skip_special_tokens=True))

## Beam Search
beam_output = transformer_model.generate(input_text,num_beams=5,max_length=100)
print(tokenizer.decode(beam_output[0],skip_special_tokens=True))

## Beam Search with n-gram Repetition removed
nr_output = transformer_model.generate(input_text,num_beams=5,no_repeat_ngram_size=5,max_length=100)
print(tokenizer.decode(nr_output[0],skip_special_tokens=True))